In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('spark').getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [ ]:
df = spark.read.csv('/FileStore/tables/data.csv',header=True,inferSchema=True)
df.show()

+----+----------+----+--------------------+---------+----------------+-----------------+-----------------+---------------+--------------------+------------+-------------+-----------+--------+----------+-----+
|Make|     Model|Year|    Engine Fuel Type|Engine HP|Engine Cylinders|Transmission Type|    Driven_Wheels|Number of Doors|     Market Category|Vehicle Size|Vehicle Style|highway MPG|city mpg|Popularity| MSRP|
+----+----------+----+--------------------+---------+----------------+-----------------+-----------------+---------------+--------------------+------------+-------------+-----------+--------+----------+-----+
| BMW|1 Series M|2011|premium unleaded ...|      335|               6|           MANUAL| rear wheel drive|              2|Factory Tuner,Lux...|     Compact|        Coupe|         26|      19|      3916|46135|
| BMW|  1 Series|2011|premium unleaded ...|      300|               6|           MANUAL| rear wheel drive|              2|  Luxury,Performance|     Compact|  Conver

In [ ]:
print(df.count())
print(df.na.drop('any').count())

11914
11812


In [ ]:
df.drop('Engine Fuel Type','Market Category','Vehicle Size','Vehicle Style').show()
slim_df = df.drop('Engine Fuel Type','Market Category','Vehicle Size','Vehicle Style').na.drop()

+----+----------+----+---------+----------------+-----------------+-----------------+---------------+-----------+--------+----------+-----+
|Make|     Model|Year|Engine HP|Engine Cylinders|Transmission Type|    Driven_Wheels|Number of Doors|highway MPG|city mpg|Popularity| MSRP|
+----+----------+----+---------+----------------+-----------------+-----------------+---------------+-----------+--------+----------+-----+
| BMW|1 Series M|2011|      335|               6|           MANUAL| rear wheel drive|              2|         26|      19|      3916|46135|
| BMW|  1 Series|2011|      300|               6|           MANUAL| rear wheel drive|              2|         28|      19|      3916|40650|
| BMW|  1 Series|2011|      300|               6|           MANUAL| rear wheel drive|              2|         28|      20|      3916|36350|
| BMW|  1 Series|2011|      230|               6|           MANUAL| rear wheel drive|              2|         28|      18|      3916|29450|
| BMW|  1 Series|201

In [ ]:
# let's use Year, Engine HP, Engine Cylinders, Trans Type, HighWay MPG, City MPG, and popularity to predict MSRP via linear regression

# first, we index the strings into numerical values
from pyspark.ml.feature import StringIndexer

index = StringIndexer(inputCols=['Make','Transmission Type'],outputCols=['makeIndex','transType'])
indexed = index.fit(slim_df).transform(slim_df)
indexed = indexed.drop('Make','Model','Transmission Type','Driven_Wheels')
indexed.show()

+----+---------+----------------+---------------+-----------+--------+----------+-----+---------+---------+
|Year|Engine HP|Engine Cylinders|Number of Doors|highway MPG|city mpg|Popularity| MSRP|makeIndex|transType|
+----+---------+----------------+---------------+-----------+--------+----------+-----+---------+---------+
|2011|      335|               6|              2|         26|      19|      3916|46135|     12.0|      1.0|
|2011|      300|               6|              2|         28|      19|      3916|40650|     12.0|      1.0|
|2011|      300|               6|              2|         28|      20|      3916|36350|     12.0|      1.0|
|2011|      230|               6|              2|         28|      18|      3916|29450|     12.0|      1.0|
|2011|      230|               6|              2|         28|      18|      3916|34500|     12.0|      1.0|
|2012|      230|               6|              2|         28|      18|      3916|31200|     12.0|      1.0|
|2012|      300|            

In [ ]:
# now, we vectorize the data

from pyspark.ml.feature import VectorAssembler as VA
ass = VA(inputCols=['Year','Engine HP','Engine Cylinders','Number of Doors','highway MPG','city mpg','Popularity','makeIndex','transType'],outputCol='vectorr')
output = ass.transform(indexed)
output.show()

+----+---------+----------------+---------------+-----------+--------+----------+-----+---------+---------+--------------------+
|Year|Engine HP|Engine Cylinders|Number of Doors|highway MPG|city mpg|Popularity| MSRP|makeIndex|transType|             vectorr|
+----+---------+----------------+---------------+-----------+--------+----------+-----+---------+---------+--------------------+
|2011|      335|               6|              2|         26|      19|      3916|46135|     12.0|      1.0|[2011.0,335.0,6.0...|
|2011|      300|               6|              2|         28|      19|      3916|40650|     12.0|      1.0|[2011.0,300.0,6.0...|
|2011|      300|               6|              2|         28|      20|      3916|36350|     12.0|      1.0|[2011.0,300.0,6.0...|
|2011|      230|               6|              2|         28|      18|      3916|29450|     12.0|      1.0|[2011.0,230.0,6.0...|
|2011|      230|               6|              2|         28|      18|      3916|34500|     12.0|

In [ ]:
output.count()

Out[28]: 11815

In [ ]:
from pyspark.ml.regression import LinearRegression as LR

final_data = output.select('vectorr','MSRP')
final_data.show(5)

+--------------------+-----+
|             vectorr| MSRP|
+--------------------+-----+
|[2011.0,335.0,6.0...|46135|
|[2011.0,300.0,6.0...|40650|
|[2011.0,300.0,6.0...|36350|
|[2011.0,230.0,6.0...|29450|
|[2011.0,230.0,6.0...|34500|
+--------------------+-----+
only showing top 5 rows



In [ ]:
train_data,test_data = final_data.randomSplit([.75,.25])
regressor = LR(featuresCol='vectorr',labelCol='MSRP')
regressor = regressor.fit(train_data)

In [ ]:
predictions = regressor.evaluate(test_data)
predictions.predictions.show()

+--------------------+----+-------------------+
|             vectorr|MSRP|         prediction|
+--------------------+----+-------------------+
|[1990.0,90.0,4.0,...|2000|-20369.708204307128|
|[1990.0,90.0,4.0,...|2000|-41114.995718979975|
|[1990.0,90.0,4.0,...|2000| -38439.31578339008|
|[1990.0,90.0,4.0,...|2000| -38439.31578339008|
|[1990.0,90.0,4.0,...|2000| -38439.31578339008|
|[1990.0,92.0,4.0,...|2000|-11605.985322570428|
|[1990.0,105.0,4.0...|2000|-19505.818571783602|
|[1990.0,105.0,4.0...|2000|-19505.818571783602|
|[1990.0,110.0,4.0...|2000| 10086.541998090222|
|[1990.0,110.0,4.0...|2000| -6461.895569418557|
|[1990.0,110.0,4.0...|2000| -42573.77116678562|
|[1990.0,110.0,4.0...|2000|  4466.884084744612|
|[1990.0,114.0,4.0...|2000| -9571.386014122749|
|[1990.0,134.0,6.0...|2098|  -5911.31672308594|
|[1990.0,135.0,6.0...|2000|-22647.791155639105|
|[1990.0,138.0,4.0...|2000| -19843.67209668248|
|[1990.0,140.0,6.0...|2000|-4054.5358711520676|
|[1990.0,140.0,6.0...|2000|-2820.7736304

In [ ]:
# so basically at the end of the day, I do not understand much about linear regression and I am using it wrong here, which results in this wildly inaccurate prediction

: 